# Behind the Pipe Line

## Tokenizer

![](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/full_nlp_pipeline.svg)

Transformer models can’t process raw text directly  
1.Splitting the input into words, subwords, or symbols (like punctuation) that are called tokens  
2.Mapping each token to an integer  
3.Adding additional inputs that may be useful to the model    

All this preprocessing needs to be done in exactly the same way as when the model was pretrained,


In [2]:
from transformers import AutoTokenizer

In [3]:
checkpoint="distilbert-base-uncased-finetuned-sst-2-english"
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

>we can directly pass our sentences to it and we’ll get back a dictionary that’s ready to feed to our model!  
>Transformer models only accept tensors as input  
> You can pass one sentence or a list of sentences, as well as specifying the type of tensors you want to get back

In [5]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]

inputs=tokenizer(raw_inputs,padding=True,truncation=True,return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


## Model

In [7]:
from transformers import AutoModel
model=AutoModel.from_pretrained(checkpoint)

>We can download our pretrained model the same way we did with our tokenizer. 🤗 

### Hidden States

This architecture contains only the base Transformer module: given some inputs, it outputs what we’ll call hidden states, also known as features. For each model input, we’ll retrieve a high-dimensional vector representing the contextual understanding of that input by the Transformer model  
like full model=transformer + head

The vector output by the Transformer module is usually large. It generally has three dimensions:

**Batch size**: The number of sequences processed at a time (2 in our example).  
**Sequence length**: The length of the numerical representation of the sequence (16 in our example).  
**Hidden size**: The vector dimension of each model input. (its the  hyper-parameter that we choose during word to vec) 

In [9]:
output=model(**inputs)

In [15]:
print(output.last_hidden_state.shape)
#(You can access the elements by attributes (like we did) 
#or by key (outputs["last_hidden_state"]), or even by index if you know exactly where the thing you are looking for is (outputs[0]).

torch.Size([2, 16, 768])


![](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/transformer_and_head.svg)

>The model heads take the high-dimensional vector of hidden states as input and project them onto a different dimension. They are usually composed of one or a few linear layers
>The embeddings layer converts each input ID in the tokenized input into a vector that represents the associated token.  
>The subsequent layers manipulate those vectors using the attention mechanism to produce the final representation of the sentences.  
>Each one designed around tackling a specific task. Here is a non-exhaustive list:
>*ForMultipleChoice  
*ForQuestionAnswering  
*ForSequenceClassification  
*ForTokenClassification

In [17]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

In [18]:
print(outputs.logits.shape)

torch.Size([2, 2])


## Post Processing

In [20]:
outputs.logits#(raw logits by last layer)

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)

In [ ]:
import torch

prediction=torch.nn.functional.softmax(outputs.logits,dim=-1)